<a href="https://colab.research.google.com/github/Roseyshi/Wine-Quality-Prediction/blob/main/Wine_Prediction_Model_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

After identifying the important features in the dataset, I will use them to identify which model will be the best to select for classification task.





In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('/content/winequalityN.csv')
features = data[['residual sugar', 'total sulfur dioxide', 'density', 'sulphates']]
data = data.drop(features, axis = 1)
data = data.dropna()
data.head()

,type,fixed acidity,volatile acidity,citric acid,chlorides,free sulfur dioxide,pH,alcohol,quality
0,white,7.0,0.27,0.36,0.045,45.0,3.00,8.8,6
1,white,6.3,0.30,0.34,0.049,14.0,3.30,9.5,6
2,white,8.1,0.28,0.40,0.050,30.0,3.26,10.1,6
3,white,7.2,0.23,0.32,0.058,47.0,3.19,9.9,6
4,white,7.2,0.23,0.32,0.058,47.0,3.19,9.9,6


In [3]:
X = data.drop(data[['quality', 'type']], axis = 1)
y = data['quality']

In [4]:
X.head()

,fixed acidity,volatile acidity,citric acid,chlorides,free sulfur dioxide,pH,alcohol
0,7.0,0.27,0.36,0.045,45.0,3.00,8.8
1,6.3,0.30,0.34,0.049,14.0,3.30,9.5
2,8.1,0.28,0.40,0.050,30.0,3.26,10.1
3,7.2,0.23,0.32,0.058,47.0,3.19,9.9
4,7.2,0.23,0.32,0.058,47.0,3.19,9.9


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
y_train.value_counts()

quality
6    1981
5    1491
7     755
4     141
8     130
3      25
9       4
Name: count, dtype: int64

We see that we have an imbalance in the number of samples in the classes of quallity. We will therefore use SMOTE for balancing.

In [8]:
#!pip install imblearn

In [9]:
from imblearn.over_sampling import SMOTE

unique, counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

# Get the count of the minority class
minority_class_count = min(class_distribution.values())

# Ensure k_neighbors is less than the number of minority samples minus one
k_neighbors = min(5, minority_class_count - 1)

smote = SMOTE(sampling_strategy='auto', k_neighbors=k_neighbors, random_state=42)

X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

{3: 25, 4: 141, 5: 1491, 6: 1981, 7: 755, 8: 130, 9: 4}


In [10]:
from collections import Counter

Counter(y_train_smote)

Counter({7: 1981, 5: 1981, 6: 1981, 4: 1981, 8: 1981, 9: 1981, 3: 1981})

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [12]:
from sklearn.model_selection import RandomizedSearchCV


# Initialize the RandomForestClassifier
rf = RandomForestClassifier()

# Define the parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [int(x) for x in np.linspace(start=100, stop=500, num=10)],
    'max_depth': [None] + [int(x) for x in np.linspace(10, 50, num=5)],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Number of iterations to perform random search
n_iter_search = 20

# Different cv values for cross-validation
cv_values = [3, 5, 10]

# Dictionary to store results for different cv values
results = {}

for cv in cv_values:
    # Initialize RandomizedSearchCV with the current cv value
    random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                       n_iter=n_iter_search, cv=cv, scoring='accuracy', n_jobs=-1, verbose=2)

    # Fit RandomizedSearchCV to the training data
    random_search.fit(X_train_smote, y_train_smote)

    # Store the best parameters and best score for the current cv value
    results[cv] = {
        'Best Parameters': random_search.best_params_,
        'Best Cross-Validation Score': random_search.best_score_,
        'Test Score': random_search.best_estimator_.score(X_test_scaled, y_test)
    }

# Print the results for each cv value
for cv, result in results.items():
    print(f"CV={cv}:")
    print("Best Parameters:", result['Best Parameters'])
    print("Best Cross-Validation Score:", result['Best Cross-Validation Score'])
    print("Test Score:", result['Test Score'])
    print("\n")




Fitting 3 folds for each of 20 candidates, totalling 60 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 10 folds for each of 20 candidates, totalling 200 fits
CV=3:
Best Parameters: {'n_estimators': 277, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 30, 'bootstrap': False}
Best Cross-Validation Score: 0.8917582769525527
Test Score: 0.6331787738279238


CV=5:
Best Parameters: {'n_estimators': 455, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None, 'bootstrap': False}
Best Cross-Validation Score: 0.9014947411061085
Test Score: 0.6275115919629057


CV=10:
Best Parameters: {'n_estimators': 455, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 50, 'bootstrap': False}
Best Cross-Validation Score: 0.9023633180085955
Test Score: 0.6383307573415765




In [14]:
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score



# Models with increased max_iter
model1 = LogisticRegression(max_iter=1000)
model2 = DecisionTreeClassifier()
model3 = SVC(probability=True, max_iter=1000)

# Voting classifier
voting = VotingClassifier(estimators=[('lr', model1), ('dt', model2), ('svc', model3)], voting='soft')

# AdaBoost classifier
base_estimator = DecisionTreeClassifier(max_depth=1, min_samples_leaf=5, criterion="entropy")
ada = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50, learning_rate=1.0)

# Gradient Boosting classifier
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)

# Parameter distributions for RandomizedSearchCV
param_dist_voting = {
    'lr__C': np.logspace(-3, 3, 7),
    'svc__C': np.logspace(-3, 3, 7),
    'dt__max_depth': [3, 5, 7]
}

param_dist_ada = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1]
}

param_dist_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# Number of parameter settings that are sampled for RandomizedSearchCV
n_iter_search = 10

# Randomized search for Voting Classifier
random_search_voting = RandomizedSearchCV(voting, param_dist_voting, n_iter=n_iter_search, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
random_search_voting.fit(X_train_smote, y_train_smote)
best_voting = random_search_voting.best_estimator_

# Randomized search for AdaBoost
random_search_ada = RandomizedSearchCV(ada, param_dist_ada, n_iter=n_iter_search, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
random_search_ada.fit(X_train_smote, y_train_smote)
best_ada = random_search_ada.best_estimator_

# Randomized search for Gradient Boosting
random_search_gb = RandomizedSearchCV(gb, param_dist_gb, n_iter=n_iter_search, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
random_search_gb.fit(X_train_smote, y_train_smote)
best_gb = random_search_gb.best_estimator_

# Predict and evaluate using the best estimators
y_pred_voting = best_voting.predict(X_test_scaled)
y_pred_ada = best_ada.predict(X_test_scaled)
y_pred_gb = best_gb.predict(X_test_scaled)

print(f'Best Voting Classifier Accuracy: {accuracy_score(y_test, y_pred_voting)}')
print(f'Best AdaBoost Accuracy: {accuracy_score(y_test, y_pred_ada)}')
print(f'Best Gradient Boosting Accuracy: {accuracy_score(y_test, y_pred_gb)}')

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Best Voting Classifier Accuracy: 0.3884595569294178
Best AdaBoost Accuracy: 0.2797527047913447
Best Gradient Boosting Accuracy: 0.6172076249356002


After hyperparameter tuning, I found out that,

CV=10:
Best Parameters: {'n_estimators': 455 'min_samples_split':5'min_samples_leaf':1, 'max_depth': 50, 'bootstrap': False}
Best Cross-Validation Score: 0.9023633180085955
Test Score: 0.6383307573415765

is the best model for this classification task. Next will be identifying how well this model will perform on classifying the quality of the wine.